In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle

In [15]:
Urban_station_lonlats = np.array([
['dongsi_aq',116.417,39.929],
['tiantan_aq',116.407,39.886],
['guanyuan_aq',116.339,39.929],
['wanshouxigong_aq',116.352,39.878],
['aotizhongxin_aq',116.397,39.982],
['nongzhanguan_aq',116.461,39.937],
['wanliu_aq',116.287,39.987],
['beibuxinqu_aq',116.174,40.09],
['zhiwuyuan_aq',116.207,40.002],
['fengtaihuayuan_aq',116.279,39.863],
['yungang_aq',116.146,39.824],
['gucheng_aq',116.184,39.914]
])
lons = np.array(Urban_station_lonlats[:,1],dtype = np.float32)
lats = np.array(Urban_station_lonlats[:,2],dtype = np.float32)
print(lons.max(),lons.min(),lons.mean())
print(lats.max(),lats.min(),lats.mean())
X_length = 93.6
Y_length = 41.2
#Change X Y to degree
Y_degree = Y_length / 111
#Consider the length of the latitude line
X_degree = X_length / (40000 * np.cos(lats.mean() / 180 * 3.1415926) / 360)
#X_degree = X_length / 111

116.461 116.146 116.30417
40.09 39.824 39.935085


In [ ]:
lat_step = np.arange(lats.mean() - Y_degree,lats.mean() + Y_degree,.1)
lon_step = np.arange(lons.mean() - X_degree,lons.mean() + X_degree,.1)
#Build the step map
for index in range(len(lat_step)):
    lat_step[index] = round(lat_step[index],1)
for index in range(len(lon_step)):
    lon_step[index] = round(lon_step[index],1)
print(lat_step)
print(lon_step)
#Get the size of point waiting to be analyzed
point_size = len(lat_step) * len(lon_step)
with open("lat_lon_range.pkl","wb") as file:
    pickle.dump((lat_step,lon_step),file)
print("ok")

In [ ]:
#Use panda to read csv
beijing_weather_data = pd.read_csv("Beijing_historical_meo_grid.csv")
print(beijing_weather_data.head())
#Get the value of beijing data
beijing_weather_data_value = beijing_weather_data.values

In [ ]:
from datetime import date
from datetime import datetime
#Build the datetime object
for index in tqdm(range(len(beijing_weather_data_value))):
    beijing_weather_data_value[index][3] = datetime.strptime(
        beijing_weather_data_value[index][3],'%Y-%m-%d %H:%M:%S')

In [ ]:
#Caculate the days between June 31 to March 1
start = date(2017,3,1)
end = date(2017,7,1)
#Set the correct array size
collection_size = point_size * (end - start).days * 24
collection = [None] * collection_size
#Select the data from March to June
def get_Data(beijing_weather_data_value,collection,collection_size):
    print(collection_size)
    cursor = 0
    for index in tqdm(range(beijing_weather_data_value.shape[0])):
        for lat in lat_step:
            for lon in lon_step:
                if (beijing_weather_data_value[index][1] == lon and 
                    beijing_weather_data_value[index][2] == lat and
                    beijing_weather_data_value[index][3].month >= 3 and
                    beijing_weather_data_value[index][3].month <= 6):
                    collection[cursor] = beijing_weather_data_value[index]
                    cursor = cursor + 1
                    if cursor == collection_size:
                        print(cursor)
                        return collection;
#Extract all the data from CSV
collection = get_Data(beijing_weather_data_value,collection,collection_size)
with open("1hour.pkl","wb") as file:
    pickle.dump(collection, file, protocol=pickle.HIGHEST_PROTOCOL)
print('ok!')

In [ ]:
#Read the data
import pickle
collection = list()
with open("1hour.pkl","rb") as file:
    collection = pickle.load(file)
print(collection[0])

In [ ]:
#Prepare datas for SVR
import pickle
from datetime import datetime
print(collection[0][3])
#collection[:,3] = datetime.strptime('%y-%m-%d %H:%M:%S',collection[:,3])
lon = np.zeros(shape = (len(collection)))
lat = np.zeros(shape = (len(collection)))
month = np.zeros(shape = (len(collection)))
date = np.zeros(shape = (len(collection)))
hour = np.zeros(shape = (len(collection)))
temp = np.zeros(shape = (len(collection)))
humid = np.zeros(shape = (len(collection)))
press = np.zeros(shape = (len(collection)))
direct = np.zeros(shape = (len(collection)))
speed = np.zeros(shape = (len(collection)))
for index in tqdm(range(len(collection))):
    lon[index] = collection[index][1]
    lat[index] = collection[index][2]
    month[index] = collection[index][3].month
    date[index] = collection[index][3].day
    hour[index] = collection[index][3].hour
    temp[index] = collection[index][4]
    humid[index] = collection[index][6]
    press[index] = collection[index][5]
    direct[index] = collection[index][7]
    speed[index] = collection[index][8]
lon = np.expand_dims(lon,axis = 1)
lat = np.expand_dims(lat,axis = 1)
month = np.expand_dims(month,axis = 1)
date = np.expand_dims(date,axis = 1)
hour = np.expand_dims(hour,axis = 1)
temp = np.expand_dims(temp,axis = 1)
humid = np.expand_dims(humid,axis = 1)
press = np.expand_dims(press,axis = 1)
direct = np.expand_dims(direct,axis = 1)
speed = np.expand_dims(speed,axis = 1)
X = np.concatenate((lon,lat,month,date,hour,temp,humid,press,direct,speed),axis = 1)
with open("1hour_prepared.pkl","wb") as file:
    pickle.dump(X, file, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
#Deserialize all the raw grid data
import pickle
with open("1hour_prepared.pkl","rb") as file:
    X = pickle.load(file)
print(X[:10])
print(X.shape)
'''0.經度(longt)
1.緯度(lat)
2.月份(month)
3.小時(hour)
4.溫度(temp)
5.濕度(humidity)
6.氣壓(pressure)
7.風向(wind_direction)
8.風速(wind_speed)'''

[[115.2   39.6    3.     1.     0.    -4.4   35.47 916.13 333.37  22.56]
 [115.2   39.7    3.     1.     0.    -5.79  39.06 905.54 332.07  25.16]
 [115.2   39.8    3.     1.     0.    -7.14  42.96 892.6  330.67  26.78]
 [115.2   39.9    3.     1.     0.    -8.44  47.44 874.98 328.6   26.46]
 [115.2   40.     3.     1.     0.    -9.73  51.93 857.35 326.49  26.17]
 [115.2   40.1    3.     1.     0.    -8.61  48.58 880.25 324.21  27.03]
 [115.2   40.2    3.     1.     0.    -7.48  45.24 903.14 322.07  27.92]
 [115.2   40.3    3.     1.     0.    -6.74  42.42 917.75 322.45  28.27]
 [115.3   39.6    3.     1.     0.    -2.54  32.41 939.33 337.13  17.22]
 [115.3   39.7    3.     1.     0.    -3.58  33.74 938.19 334.49  21.71]]
(515328, 10)


'0.經度(longt)\n1.緯度(lat)\n2.月份(month)\n3.小時(hour)\n4.溫度(temp)\n5.濕度(humidity)\n6.氣壓(pressure)\n7.風向(wind_direction)\n8.風速(wind_speed)'

# SVR analysis

In [ ]:
#SVR analysis
'''X => SVR輸入向量的numpy array，資料必須照著下面的順序:
0.經度(longt)
1.緯度(lat)
2.月份(month)
3.小時(hour)
4.溫度(temp)
5.濕度(humidity)
6.氣壓(pressure)
7.風向(wind_direction)
8.風速(wind_speed)
Y => PM2.5的預測值'''
import pickle
from sklearn.svm import SVR
from sklearn.externals import joblib
import numpy as np

svr = joblib.load('SVR_beijing.pkl')
X = np.array(X, dtype=np.float64)
X_mu = np.array([116.40616279,39.98908696,4.45972151,14.88423765,11.40249715,17.0021192,33.65642229,996.1066705,208.34196307,10.85087688])
X_sigma = np.array([0.28161015,0.22815651,1.14854307,8.88446275,6.90495716,8.44671392,19.41044621,19.83802731,96.65489494,6.85870324])
Y_mu = 59.541867360598516
Y_sigma = 61.21714698899477

Xs = (X - X_mu) / X_sigma #對要輸入的X向量標準化
Y_pdt = svr.predict(Xs) #將標準化後的X向量放入SVR預測
Y = Y_pdt * Y_sigma + Y_mu #對SVR得到的預測值反標準化
print(Y[:10])

In [ ]:
import pickle as pk
with open("1_hr_pm25.pkl","wb") as file:
    pk.dump(Y, file, protocol=pickle.HIGHEST_PROTOCOL)

# Preprocessing

In [26]:
import pickle as pk
import numpy as np
from datetime import date
#load all the prepared data for SVR(to add feature to )
with open("1hour_prepared.pkl","rb") as file:
    X = pickle.load(file)
#load all the SVR data
with open("1_hr_pm25.pkl","rb") as file:
    Y = pk.load(file)
#load the ercise point of latitude and longitude
with open("lat_lon_range.pkl","rb") as file:
    lat_step, lon_step = pk.load(file)
#Y transform to [date][lon][lat]
date_count = (date(2017,7,1) - date(2017,3,1)).days
Y = np.reshape(Y, (24 * date_count,len(lon_step),len(lat_step),1))
temp = np.reshape(X[:,5],Y.shape)
Y = np.concatenate((Y,temp),axis = 3)
#Get the longitude of the target site
target_lon = np.argwhere(lon_step == 39.9)
#Get the latitude of the target site
target_lat = np.argwhere(lat_step == 116.3)
#There  are 256 objects in pickle file and there are 10 pickle files
validation_size = len(Y) - 2560 
#Dump validation data
with open("1hour_pm25_validation_array.pkl","wb") as file:
    pk.dump((Y[-validation_size - 1:-1],Y[-validation_size:,target_lon,target_lat,0]),file)
#Dump testing data
#Assume that batch size is 256
for index in range(1, int(len(Y) / 256)):
    with open("batch_number_" + str(index) + ".pkl","wb") as file:
        if 256 * index >= len(Y):
            end_batch = len(Y) - 1
        else:
            end_batch = 256 * index
        pk.dump((Y[256 * (index - 1):end_batch],#Testing batch
                 Y[256 * (index - 1) + 1:end_batch + 1 ,target_lon,target_lat,0]),file)#Testing label
print("ok!")

ok!


# Start building network

In [27]:
import tensorflow as tf
import numpy as np
import pickle as pk
import os

In [28]:
#print(validation_size)
batch = list()
batch_label = list()
#Serialize all the batch
for index in range(1,11):
    with open("batch_number_" + str(index) + ".pkl","rb") as file:
        batch_read, batch_label_read = pk.load(file)
        batch.append(batch_read)
        batch_label.append(batch_label_read)
#Serialize the area 
with open("lat_lon_range.pkl","rb") as file:
    lat_step, lon_step = pk.load(file)

with open("1hour_pm25_validation_array.pkl","rb") as file:
    X_validation, Y_validation = pk.load(file)
print("================")
print(X_validation.shape)
print(Y_validation.shape)
print("================")
#print(X_validation[-1])
#print(Y_validation[-2])
batch = np.array(batch)
batch_label = np.array(batch_label)
print(batch.shape)
print(batch_label.shape)
print("================")

(368, 22, 8, 2)
(368, 0, 1)
(10, 256, 22, 8, 2)
(10, 256, 0, 1)


In [5]:
#Normalize input data
def normalize(x):
    norm = (x - x.min())/(x.max() - x.min())
    return norm

In [6]:
def Build_CNN(x):
    CNN = tf.layers.conv2d(inputs = x,filters = 64,kernel_size = 3,padding = 'SAME',activation = tf.nn.relu)
    CNN = tf.nn.relu(CNN)
    CNN = tf.layers.conv2d(inputs = CNN, filters = 64,kernel_size = 3,padding = 'SAME',activation = tf.nn.relu)
    CNN = tf.nn.relu(CNN)
    CNN = tf.contrib.layers.flatten(CNN)
    CNN = tf.contrib.layers.fully_connected(CNN,128)
    CNN = tf.nn.relu(CNN)
    CNN = tf.contrib.layers.fully_connected(CNN,128)
    CNN = tf.nn.relu(CNN)
    CNN = tf.contrib.layers.fully_connected(CNN,1)
    return CNN

In [7]:
def print_loss(session,validation_batch,validation_label,loss):
    loss = session.run(loss,feed_dict = {x:validation_batch,y:validation_label})
    print("|loss:{:.7f}".format(loss),"|")
    pass

In [8]:
#Reset the graph
tf.reset_default_graph()
#Input of the network
x = tf.placeholder(tf.float32,shape = (None,len(lon_step),len(lat_step),1), name = 'x')
y = tf.placeholder(tf.float32,shape = (None),name = 'y')
#Build the network
logits = Build_CNN(x)
# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

#Define Loss and optimizer
a = tf.abs(tf.subtract(y,logits))
b = tf.add(y,logits)
loss = tf.scalar_mul(2, tf.reduce_mean(tf.divide(a, b)))
#MSE version 
#loss = tf.sqrt(tf.reduce_mean(tf.losses.mean_squared_error(labels = y,predictions = logits)))
optimizer = tf.train.AdamOptimizer().minimize(loss)


In [9]:
#Loss test
import numpy as np
def smape(actual, predicted):
    a = np.abs(np.array(actual) - np.array(predicted))
    print(a)
    b = np.array(actual) + np.array(predicted)
    print(b)
    print(np.divide(a, b, out=np.zeros_like(a), where=b!=0, casting='unsafe'))
    return 2 * np.mean(np.divide(a, b))
a = np.array([1,2,3,4])
b = np.array([2,99,4,5])
print(smape(a,b))

[ 1 97  1  1]
[  3 101   7   9]
[0 0 0 0]
0.7738488134527738


In [12]:
epoch = 10000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epoch):
        for batch_num in range(0,10):
            sess.run(optimizer,feed_dict = {x:batch[batch_num],y:batch_label[batch_num]})
            loss_value = sess.run(loss,feed_dict = {x:X_validation,y:Y_validation})
        print("|loss:",loss_value,"|")

|loss: 0.48209846 |
|loss: 0.442506 |
|loss: 0.48542574 |
|loss: 0.4442621 |
|loss: 0.43022144 |
|loss: 0.43439505 |
|loss: 0.4602072 |
|loss: 0.44819877 |
|loss: 0.4368088 |
|loss: 0.43895948 |
|loss: 0.443519 |
|loss: 0.45190883 |
|loss: 0.45693353 |
|loss: 0.4464655 |
|loss: 0.44241923 |
|loss: 0.42749986 |
|loss: 0.4224245 |
|loss: 0.4315503 |
|loss: 0.4902531 |
|loss: 0.4634153 |
|loss: 0.4490166 |
|loss: 0.43608442 |
|loss: 0.4273219 |
|loss: 0.43310893 |
|loss: 0.44861692 |
|loss: 0.4803127 |
|loss: 0.43568167 |
|loss: 0.42385647 |
|loss: 0.42724696 |
|loss: 0.4691205 |
|loss: 0.46190044 |
|loss: 0.41920206 |
|loss: 0.42598578 |
|loss: 0.46230626 |
|loss: 0.46011397 |
|loss: 0.4175265 |
|loss: 0.42792022 |
|loss: 0.4629006 |
|loss: 0.45297253 |
|loss: 0.41959438 |
|loss: 0.43478912 |
|loss: 0.4613104 |
|loss: 0.44690162 |
|loss: 0.42054546 |
|loss: 0.43664443 |
|loss: 0.4628848 |
|loss: 0.4405825 |
|loss: 0.42173812 |
|loss: 0.43500656 |
|loss: 0.45606863 |
|loss: 0.43588084 |
|

|loss: 0.44169262 |
|loss: 0.41532955 |
|loss: 0.42684665 |
|loss: 0.44032416 |
|loss: 0.42327237 |
|loss: 0.42122012 |
|loss: 0.44104728 |
|loss: 0.42437032 |
|loss: 0.43179652 |
|loss: 0.4334602 |
|loss: 0.4217238 |
|loss: 0.4352814 |
|loss: 0.43542665 |
|loss: 0.42090055 |
|loss: 0.43330973 |
|loss: 0.43381423 |
|loss: 0.4218428 |
|loss: 0.43466884 |
|loss: 0.43077737 |
|loss: 0.42353576 |
|loss: 0.43598378 |
|loss: 0.42894527 |
|loss: 0.4227918 |
|loss: 0.43369284 |
|loss: 0.4298316 |
|loss: 0.42484707 |
|loss: 0.42989707 |
|loss: 0.4299608 |
|loss: 0.42394742 |
|loss: 0.429701 |
|loss: 0.43271863 |
|loss: 0.42354476 |
|loss: 0.42757022 |
|loss: 0.43514034 |
|loss: 0.42402688 |
|loss: 0.42355826 |
|loss: 0.43747035 |
|loss: 0.42834768 |
|loss: 0.41977918 |
|loss: 0.4419042 |
|loss: 0.42942005 |
|loss: 0.41828918 |
|loss: 0.4404003 |
|loss: 0.42671415 |
|loss: 0.41955823 |
|loss: 0.43810242 |
|loss: 0.43005508 |
|loss: 0.41555163 |
|loss: 0.44179192 |
|loss: 0.4383844 |
|loss: 0.418

|loss: 0.4238747 |
|loss: 0.42583868 |
|loss: 0.42863658 |
|loss: 0.4241806 |
|loss: 0.42942256 |
|loss: 0.4324875 |
|loss: 0.42307878 |
|loss: 0.42611423 |
|loss: 0.43608034 |
|loss: 0.42361796 |
|loss: 0.42310262 |
|loss: 0.4348513 |
|loss: 0.4313558 |
|loss: 0.4194613 |
|loss: 0.43316555 |
|loss: 0.43209538 |
|loss: 0.41931283 |
|loss: 0.4318704 |
|loss: 0.434872 |
|loss: 0.42191568 |
|loss: 0.43337587 |
|loss: 0.4300784 |
|loss: 0.42744446 |
|loss: 0.4277284 |
|loss: 0.4346154 |
|loss: 0.4278078 |
|loss: 0.42635608 |
|loss: 0.43018737 |
|loss: 0.42153406 |
|loss: 0.4231338 |
|loss: 0.4357342 |
|loss: 0.41968822 |
|loss: 0.42201647 |
|loss: 0.43716446 |
|loss: 0.426059 |
|loss: 0.41878706 |
|loss: 0.43289664 |
|loss: 0.43398315 |
|loss: 0.4212725 |
|loss: 0.4363912 |
|loss: 0.4328189 |
|loss: 0.42982885 |
|loss: 0.43464035 |
|loss: 0.42548344 |
|loss: 0.42018145 |
|loss: 0.43518916 |
|loss: 0.4231032 |
|loss: 0.42028826 |
|loss: 0.4354304 |
|loss: 0.42741418 |
|loss: 0.42019284 |
|l

|loss: 0.42638645 |
|loss: 0.4198851 |
|loss: 0.4280763 |
|loss: 0.43034974 |
|loss: 0.41880244 |
|loss: 0.42582798 |
|loss: 0.4311199 |
|loss: 0.42115623 |
|loss: 0.4227794 |
|loss: 0.43076032 |
|loss: 0.4234062 |
|loss: 0.4204387 |
|loss: 0.42970636 |
|loss: 0.42766106 |
|loss: 0.4198268 |
|loss: 0.42729518 |
|loss: 0.4315343 |
|loss: 0.4223341 |
|loss: 0.4231769 |
|loss: 0.43054157 |
|loss: 0.4246948 |
|loss: 0.42057034 |
|loss: 0.43149647 |
|loss: 0.42935562 |
|loss: 0.42264947 |
|loss: 0.4290166 |
|loss: 0.42971253 |
|loss: 0.42113844 |
|loss: 0.42621532 |
|loss: 0.4350624 |
|loss: 0.42332917 |
|loss: 0.41986325 |
|loss: 0.42875782 |
|loss: 0.4319652 |
|loss: 0.4184304 |
|loss: 0.42744076 |
|loss: 0.4356426 |
|loss: 0.42164746 |
|loss: 0.42554858 |
|loss: 0.43223402 |
|loss: 0.42316598 |
|loss: 0.42546177 |
|loss: 0.43196693 |
|loss: 0.42151955 |
|loss: 0.4252247 |
|loss: 0.43491814 |
|loss: 0.4264912 |
|loss: 0.42284146 |
|loss: 0.43223044 |
|loss: 0.4308755 |
|loss: 0.42084467 |

|loss: 0.43953723 |
|loss: 0.41893154 |
|loss: 0.4220505 |
|loss: 0.43131077 |
|loss: 0.4245094 |
|loss: 0.4225753 |
|loss: 0.43312117 |
|loss: 0.42157966 |
|loss: 0.41821223 |
|loss: 0.42876628 |
|loss: 0.43035078 |
|loss: 0.41938904 |
|loss: 0.42626926 |
|loss: 0.4313296 |
|loss: 0.4201416 |
|loss: 0.4265028 |
|loss: 0.429788 |
|loss: 0.42018706 |
|loss: 0.42877123 |
|loss: 0.42804453 |
|loss: 0.42000023 |
|loss: 0.4306111 |
|loss: 0.42229378 |
|loss: 0.42030263 |
|loss: 0.42816517 |
|loss: 0.4242041 |
|loss: 0.419786 |
|loss: 0.42858684 |
|loss: 0.42486542 |
|loss: 0.42012507 |
|loss: 0.4252034 |
|loss: 0.42563123 |
|loss: 0.42169136 |
|loss: 0.42335957 |
|loss: 0.4253518 |
|loss: 0.4224393 |
|loss: 0.42280146 |
|loss: 0.42580652 |
|loss: 0.42346194 |
|loss: 0.4218842 |
|loss: 0.42550823 |
|loss: 0.42445618 |
|loss: 0.4213722 |
|loss: 0.42372563 |
|loss: 0.42606944 |
|loss: 0.42050475 |
|loss: 0.42240703 |
|loss: 0.42658016 |
|loss: 0.42216682 |
|loss: 0.42101294 |
|loss: 0.4261378 

...

|loss: 0.42596522 |
|loss: 0.43315193 |
|loss: 0.4237394 |
|loss: 0.42750257 |
|loss: 0.4323942 |
|loss: 0.42510447 |
|loss: 0.42596778 |
|loss: 0.43195727 |
|loss: 0.42604506 |
|loss: 0.4246666 |
|loss: 0.43148348 |
|loss: 0.42523703 |
|loss: 0.42393798 |
|loss: 0.4295391 |
|loss: 0.42436892 |
|loss: 0.42533565 |
|loss: 0.4285722 |
|loss: 0.42480242 |
|loss: 0.42503312 |
|loss: 0.43163303 |
|loss: 0.42428455 |
|loss: 0.42523187 |
|loss: 0.42868984 |
|loss: 0.4232111 |
|loss: 0.42782602 |
|loss: 0.43141368 |
|loss: 0.4248458 |
|loss: 0.42590195 |
|loss: 0.42811435 |
|loss: 0.42365158 |
|loss: 0.42566058 |
|loss: 0.42930388 |
|loss: 0.4262665 |
|loss: 0.42564246 |
|loss: 0.42808142 |
|loss: 0.4255831 |
|loss: 0.4254445 |
|loss: 0.42783922 |
|loss: 0.42677245 |
|loss: 0.42574355 |
|loss: 0.42582065 |
|loss: 0.4286133 |
|loss: 0.4256311 |
|loss: 0.42510724 |
|loss: 0.42725626 |
|loss: 0.4276399 |
|loss: 0.424911 |
|loss: 0.42510864 |
|loss: 0.4291696 |
|loss: 0.4264305 |
|loss: 0.42476878

|loss: 0.430937 |
|loss: 0.42441222 |
|loss: 0.4223808 |
|loss: 0.4298111 |
|loss: 0.42555365 |
|loss: 0.42257398 |
|loss: 0.4309386 |
|loss: 0.42469555 |
|loss: 0.42299902 |
|loss: 0.42802194 |
|loss: 0.42535806 |
|loss: 0.4226329 |
|loss: 0.42870274 |
|loss: 0.42603478 |
|loss: 0.42366573 |
|loss: 0.42878243 |
|loss: 0.4249396 |
|loss: 0.4236452 |
|loss: 0.4300877 |
|loss: 0.42498392 |
|loss: 0.42311132 |
|loss: 0.4282727 |
|loss: 0.4243418 |
|loss: 0.4236049 |
|loss: 0.42946962 |
|loss: 0.42401454 |
|loss: 0.42363897 |
|loss: 0.42825627 |
|loss: 0.42460483 |
|loss: 0.42569327 |
|loss: 0.42910492 |
|loss: 0.42440426 |
|loss: 0.4233823 |
|loss: 0.42998746 |
|loss: 0.42445856 |
|loss: 0.4236651 |
|loss: 0.4312145 |
|loss: 0.4245488 |
|loss: 0.4234893 |
|loss: 0.4295283 |
|loss: 0.42439288 |
|loss: 0.42396325 |
|loss: 0.4303955 |
|loss: 0.42442885 |
|loss: 0.423007 |
|loss: 0.42845434 |
|loss: 0.4238785 |
|loss: 0.42354748 |
|loss: 0.42913872 |
|loss: 0.425003 |
|loss: 0.42408746 |
|los

|loss: 0.4291513 |
|loss: 0.42644644 |
|loss: 0.42415679 |
|loss: 0.42631528 |
|loss: 0.42642707 |
|loss: 0.42488393 |
|loss: 0.4241284 |
|loss: 0.4279893 |
|loss: 0.42645463 |
|loss: 0.42396754 |
|loss: 0.42737037 |
|loss: 0.42670524 |
|loss: 0.42479077 |
|loss: 0.4237043 |
|loss: 0.4285446 |
|loss: 0.42643693 |
|loss: 0.42399687 |
|loss: 0.4272254 |
|loss: 0.42669016 |
|loss: 0.42436442 |
|loss: 0.42415375 |
|loss: 0.42872652 |
|loss: 0.42685476 |
|loss: 0.4233773 |
|loss: 0.42760968 |
|loss: 0.42625776 |
|loss: 0.42365515 |
|loss: 0.4240854 |
|loss: 0.42963728 |
|loss: 0.42564785 |
|loss: 0.4231864 |
|loss: 0.428938 |
|loss: 0.4248975 |
|loss: 0.42309722 |
|loss: 0.42626148 |
|loss: 0.42891046 |
|loss: 0.42445332 |
|loss: 0.4246047 |
|loss: 0.42855218 |
|loss: 0.4254416 |
|loss: 0.42419043 |
|loss: 0.42732003 |
|loss: 0.42727903 |
|loss: 0.42402598 |
|loss: 0.42354047 |
|loss: 0.42859343 |
|loss: 0.42524183 |
|loss: 0.4233609 |
|loss: 0.4279402 |
|loss: 0.42593727 |
|loss: 0.4240242

|loss: 0.42744783 |
|loss: 0.4276053 |
|loss: 0.42576686 |
|loss: 0.4250916 |
|loss: 0.42939758 |
|loss: 0.4272125 |
|loss: 0.42376277 |
|loss: 0.42553914 |
|loss: 0.42926955 |
|loss: 0.42554617 |
|loss: 0.42372957 |
|loss: 0.4298304 |
|loss: 0.42652044 |
|loss: 0.4238607 |
|loss: 0.42711425 |
|loss: 0.42728055 |
|loss: 0.4260584 |
|loss: 0.42587003 |
|loss: 0.4289885 |
|loss: 0.42682445 |
|loss: 0.42410094 |
|loss: 0.42654973 |
|loss: 0.42972735 |
|loss: 0.42547524 |
|loss: 0.42459413 |
|loss: 0.42966914 |
|loss: 0.42726994 |
|loss: 0.42576468 |
|loss: 0.42633316 |
|loss: 0.4292817 |
|loss: 0.4268642 |
|loss: 0.42477047 |
|loss: 0.4273889 |
|loss: 0.42764413 |
|loss: 0.42663938 |
|loss: 0.42615113 |
|loss: 0.42950186 |
|loss: 0.42617062 |
|loss: 0.42495292 |
|loss: 0.42766535 |
|loss: 0.42854053 |
|loss: 0.42434865 |
|loss: 0.42446446 |
|loss: 0.43060932 |
|loss: 0.42681625 |
|loss: 0.42429495 |
|loss: 0.42860994 |
|loss: 0.42792708 |
|loss: 0.4245145 |
|loss: 0.42561796 |
|loss: 0.43

KeyboardInterrupt: 